Make a list of the color, lightcurve stretch, color stretch, peak time and magnitudes for every band,
peak magnitude's error and save the plots of flux in relation to time for every supernovae

In [ ]:
import sncosmo
import os
import math
import warnings
import numpy as np

input_folder_path = './input/'

all_parameters = []

failed_files = []

output_folder_path = './list/'

failed_folder_path = './failed_fits/'

# Loop through all the .txt files in the input folder
for i, input_file_name in enumerate(os.listdir(input_folder_path)):
    if i == 3700:
        break
    if input_file_name.endswith('.txt'):
        input_file_path = os.path.join(input_folder_path, input_file_name)

        # Open the input file for reading
        try:
            with open(input_file_path, 'r') as f:
                first_line = f.readline()
                z = float(first_line.split()[0])

                data = sncosmo.read_lc(f)

                # Create a model
                model = sncosmo.Model(source='salt3-nir')
                model.set(z=z)
                with warnings.catch_warnings():
                    warnings.filterwarnings("ignore", category=RuntimeWarning)
                    result, fitted_model = sncosmo.fit_lc(
                        data, model,
                        ['t0', 'x0', 'x1', 'c']) 
                    
                    # Create a directory to save the plots
                    plots_folder_path = os.path.join(input_folder_path, 'plots')
                    if not os.path.exists(plots_folder_path):
                        os.makedirs(plots_folder_path)

                    plot_file_name = input_file_name.rstrip('.txt') + '.png'
                    plot_file_path = os.path.join(plots_folder_path, plot_file_name)
                    fig = sncosmo.plot_lc(data, model=fitted_model, errors=result.errors)
                    fig.savefig(plot_file_path)

                    # Peak magnitudes
                    ztfg = fitted_model.source_peakmag('ztfg', 'ab')
                    ztfi = fitted_model.source_peakmag('ztfi', 'ab')
                    ztfr = fitted_model.source_peakmag('ztfr', 'ab')
                    atlasc = fitted_model.source_peakmag('atlasc', 'ab')
                    atlaso = fitted_model.source_peakmag('atlaso', 'ab')
                    massj = fitted_model.source_peakmag('2massj', 'ab')
                    massh = fitted_model.source_peakmag('2massh', 'ab')
                                        
                    # Peak magnitude's error
                    magerr = 2.5 / math.log(10) * result.errors['x0'] / result.parameters[2]
                    

                # Add the parameters to the list
                all_parameters.append([input_file_name.rstrip('.txt')] + list(result.parameters) 
                                      + [result.errors['t0']] + [result.errors['x0']] 
                                      + [result.errors['x1']] + [result.errors['c']] 
                                      + [ztfg] + [ztfi] + [ztfr] + [atlasc] + [atlaso] + [massj] + [massh] + [magerr])


                # Write the parameters to a txt file in the output folder for successful fits
                output_file_path = os.path.join(output_folder_path, 'salt3_nir_fitted.txt')
                with open(output_file_path, 'w') as f:
                    f.write("name                z          t0                    x0                    x1                     c                  t0_err                  x0_err             x1_err                 c_err           peakmag_ztfg           peakmag_ztfi           peakmag_ztfr          peakmag_atlasc           peakmag_atlaso           peakmag_massj            peakmag_massh           mag_err\n")
                    for parameters in all_parameters:
                        f.write('\t'.join(str(p) for p in parameters) + '\n')
                        
        except Exception as e:
            failed_files.append(input_file_name)
            output_file_path = os.path.join(failed_folder_path, 'failed_fits_salt3_nir_fitted.txt')
            with open(output_file_path, 'a') as f:
                f.write("Error reading file " + input_file_name + ": " + str(e) + "\n")
                
# Write the list of failed files to a txt file in the failed folder
output_file_path = os.path.join(failed_folder_path, 'failed_files_salt3_nir_fitted.txt')
with open(output_file_path, 'w') as f:
    for failed_file in failed_files:
        f.write(failed_file + "\n")
